<a href="https://colab.research.google.com/github/Rebecca-kve/FreeCodeCamp/blob/main/Machine%20Learning%20with%20Python/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:

from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Convert categorical columns to numerical using OneHotEncoding
categorical_columns = ['sex', 'smoker', 'region']
numerical_columns = ['age', 'bmi', 'children']

# Separate features and labels
X = dataset.drop('expenses', axis=1)
y = dataset['expenses']

# Build a preprocessor to handle numerical scaling and categorical encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),  # Scale numerical columns
        ('cat', OneHotEncoder(), categorical_columns)  # One-hot encode categorical columns
    ])

# Example: Add an interaction term between BMI and smoker status
dataset['bmi_smoker_interaction'] = dataset['bmi'] * (dataset['smoker'] == 'yes').astype(int)


# Apply the transformations to the features
X_transformed = preprocessor.fit_transform(X)

# Step 4: Split the dataset into training and test sets (80% train, 20% test)
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X_transformed, y, test_size=0.2, random_state=42)


In [ ]:

def build_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=[train_dataset.shape[1]]),         # Input layer
        layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        layers.Dropout(0.2),                                    # Dropout to prevent overfitting
        layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        layers.Dense(1)                                         # Output layer for regression
    ])

    # Compile the model with a reduced learning rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)   # Reduced learning rate
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae', 'mse'])

    return model


model = build_model()

# Add early stopping to stop training when validation loss stops improving
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Train the model
history = model.fit(
    train_dataset,
    train_labels,
    epochs=200,                  # Increase the number of epochs
    validation_split=0.2,        # Use 20% of the training data for validation
    verbose=1,
    callbacks=[early_stop]       # Early stopping to avoid overfitting
)

In [ ]:
# Step 7: Plot the training progress (Optional, but helps visualize training)
plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error [Expenses]')
plt.plot(history.epoch, np.array(history.history['mae']), label='Train Loss')
plt.plot(history.epoch, np.array(history.history['val_mae']), label='Val Loss')
plt.legend()
plt.show()

# Check if the Mean Absolute Error is less than 3500
print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
